In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [20]:
# Load the data from the data folder
bus = pd.read_csv('../Data/B (power transfer factor of each bus to each line).csv')
w_capacity = pd.read_csv('../Data/Capacity of wind farms.csv')
loads = pd.read_csv('../Data/Loads.csv')
max_prod = pd.read_csv('../Data/Maximum production of generating units.csv')
min_prod = pd.read_csv('../Data/Minimum production of generating units.csv')
min_down_time = pd.read_csv('../Data/Minimum down time of generating units.csv')
min_up_time = pd.read_csv('../Data/Minimum up time of generating units.csv')
prod_cost = pd.read_csv('../Data/Production cost of generating units.csv')
ramp_rate = pd.read_csv('../Data/Ramping rate of generating units.csv')
start_up_cost = pd.read_csv('../Data/Start-up cost of generating units.csv')
transmission_cap = pd.read_csv('../Data/Transmission capacity of lines.csv')
